In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plot
import numpy as np
import datetime as dt
pd.set_option('display.max_columns', None)

%matplotlib inline

In [2]:
gc_members = pd.read_csv('../data/NSS_GreatCare_Members.csv', low_memory=False)

In [3]:
ip_admin = pd.read_csv('../data/NSS_Hospital_Inpatient_Admissions.csv', low_memory=False)

In [4]:
pc_visits = pd.read_csv('../data/NSS_PrimaryCare_Office_Visits.csv', low_memory=False)

In [5]:
# pc_visits.head()

In [6]:
# ip_admin.head()

In [7]:
ip_admin['Encounter Start Date'] = pd.to_datetime(ip_admin['Encounter Start Date'], format='%Y-%m-%d')
ip_admin['Encounter End Date'] = pd.to_datetime(ip_admin['Encounter End Date'], format='%Y-%m-%d')

In [8]:
ip_admin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 23 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   Encounter ID                                         6213 non-null   object        
 1   Patient ID                                           6213 non-null   object        
 2   Encounter Primary Payor ID                           6213 non-null   object        
 3   Encounter Primary Plan ID                            6213 non-null   object        
 4   Facility Provider ID                                 6213 non-null   object        
 5   Attending Provider ID                                5884 non-null   object        
 6   Attending Provider NPI                               5884 non-null   float64       
 7   Facility Provider NPI                                6213 non-null   object        
 8 

In [9]:
# for i in ip_admin['Patient ID']:
#     if i == i:
#         ip_admin['Encounter End Date'][0] - ip_admin['Encounter Start Date'][1]
#     print(i)

In [10]:
# ip_admin['time_lapse'] = ip_admin['Encounter End Date'] - ip_admin['Encounter Start Date']

In [11]:
ip_admin = ip_admin.sort_values(['Patient ID','Encounter Start Date', 'Encounter Start Date'], ascending = (False, True, True))

In [12]:
ip_admin1 = ip_admin[['Patient ID', 'Encounter Start Date', 'Encounter End Date']]

In [13]:
ip_admin1

,Patient ID,Encounter Start Date,Encounter End Date
6212,DS1|9996,2010-03-28,2010-03-31
6210,DS1|9994,2015-11-21,2015-11-23
6211,DS1|9994,2015-11-24,2015-11-27
6209,DS1|9981,2014-08-07,2014-08-10
6208,DS1|9979,2015-02-06,2015-02-11
...,...,...,...
3,DS1|10101,2012-12-06,2012-12-09
2,DS1|10073,2012-06-13,2012-06-21
1,DS1|10043,2014-01-05,2014-01-12
0,DS1|10033,2013-01-12,2013-01-14


In [14]:
# ip_admin1['diff'] = ip_admin1.groupby('Patient ID')['Encounter Start Date'].diff() / np.timedelta64(1, 'D')
# ip_admin1['diff'] = ip_admin1['diff'].fillna(0)
# ip_admin1['diff_end'] = ip_admin1.groupby('Patient ID')['Encounter End Date'].diff() / np.timedelta64(1, 'D')
# ip_admin1['diff_end'] = ip_admin1['diff_end'].fillna(0)
# ip_admin1

In [15]:
ip_admin1['Previous End Date'] = (ip_admin1.groupby('Patient ID')['Encounter End Date'].shift(1))
ip_admin1['Days Since Last Visit'] = ip_admin1['Encounter Start Date'] - ip_admin1['Previous End Date']
ip_admin1 = ip_admin1[ip_admin1['Days Since Last Visit'].notnull()]
ip_admin_30 = ip_admin1[ip_admin1['Days Since Last Visit'] <= '30 days']
ip_admin_under_30 = ip_admin1[ip_admin1['Days Since Last Visit'] < '30 days']
ip_admin_30

/Users/richietownsend/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/richietownsend/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Patient ID,Encounter Start Date,Encounter End Date,Previous End Date,Days Since Last Visit
6211,DS1|9994,2015-11-24,2015-11-27,2015-11-23,1 days
6202,DS1|9943,2013-10-25,2013-10-28,2013-10-16,9 days
6199,DS1|9935,2011-01-25,2011-01-26,2011-01-24,1 days
6197,DS1|9935,2011-01-26,2011-02-07,2011-01-26,0 days
6185,DS1|9809,2013-01-20,2013-01-24,2013-01-11,9 days
...,...,...,...,...,...
38,DS1|10451,2013-05-01,2013-05-09,2013-04-26,5 days
40,DS1|10451,2013-07-26,2013-08-03,2013-07-01,25 days
41,DS1|10451,2013-08-31,2013-09-02,2013-08-03,28 days
43,DS1|10451,2013-12-17,2013-12-19,2013-11-20,27 days


In [16]:
rows = len(ip_admin_30)
rows

755

In [17]:
ip_admin_30

,Patient ID,Encounter Start Date,Encounter End Date,Previous End Date,Days Since Last Visit
6211,DS1|9994,2015-11-24,2015-11-27,2015-11-23,1 days
6202,DS1|9943,2013-10-25,2013-10-28,2013-10-16,9 days
6199,DS1|9935,2011-01-25,2011-01-26,2011-01-24,1 days
6197,DS1|9935,2011-01-26,2011-02-07,2011-01-26,0 days
6185,DS1|9809,2013-01-20,2013-01-24,2013-01-11,9 days
...,...,...,...,...,...
38,DS1|10451,2013-05-01,2013-05-09,2013-04-26,5 days
40,DS1|10451,2013-07-26,2013-08-03,2013-07-01,25 days
41,DS1|10451,2013-08-31,2013-09-02,2013-08-03,28 days
43,DS1|10451,2013-12-17,2013-12-19,2013-11-20,27 days


In [18]:
ip_admin1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1512 entries, 6211 to 7
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   Patient ID             1512 non-null   object         
 1   Encounter Start Date   1512 non-null   datetime64[ns] 
 2   Encounter End Date     1512 non-null   datetime64[ns] 
 3   Previous End Date      1512 non-null   datetime64[ns] 
 4   Days Since Last Visit  1512 non-null   timedelta64[ns]
dtypes: datetime64[ns](3), object(1), timedelta64[ns](1)
memory usage: 70.9+ KB


In [19]:
ip_admin1[ip_admin1['Days Since Last Visit'].notnull()]

,Patient ID,Encounter Start Date,Encounter End Date,Previous End Date,Days Since Last Visit
6211,DS1|9994,2015-11-24,2015-11-27,2015-11-23,1 days
6202,DS1|9943,2013-10-25,2013-10-28,2013-10-16,9 days
6203,DS1|9943,2014-10-28,2014-10-30,2013-10-28,365 days
6199,DS1|9935,2011-01-25,2011-01-26,2011-01-24,1 days
6197,DS1|9935,2011-01-26,2011-02-07,2011-01-26,0 days
...,...,...,...,...,...
36,DS1|10451,2014-06-29,2014-06-29,2014-05-29,31 days
21,DS1|10291,2011-02-24,2011-03-05,2010-05-15,285 days
18,DS1|10253,2010-06-02,2010-06-06,2010-04-16,47 days
17,DS1|10253,2011-10-15,2011-11-13,2010-06-06,496 days


In [20]:
#Using .shift
# ip_admin1['lapse'] = (ip_admin1['Encounter End Date'] - ip_admin1['Encounter Start Date'].shift(1))

Brenda Code

In [21]:
# #make df(patient, start date, end date)
# one = adm[['Patient ID','Encounter Start Date','Encounter End Date']]
# #set datetime
# one['Encounter Start Date'] = pd.to_datetime(one['Encounter Start Date'], format='%Y-%m-%d')
# one['Encounter End Date'] = pd.to_datetime(one['Encounter End Date'], format='%Y-%m-%d')
# #order by patient then start date
# one = one.sort_values(['Patient ID',"Encounter Start Date", "Encounter End Date"], ascending = (False, True, True))
# #if difference between Encounter End Date and Encounter Start Date < 30
# one['diff'] = one.groupby('Patient ID')['Encounter Start Date'].diff() / np.timedelta64(1, 'D')
# one['diff'] = one['diff'].fillna(0)
# one['diff_end'] = one.groupby('Patient ID')['Encounter End Date'].diff() / np.timedelta64(1, 'D')
# one['diff_end'] = one['diff_end'].fillna(0)
# one